## Import Autophagy Package

In [ ]:
from cr_autophagy_pyo3 import *

## Define Simulation Settings
These settings are predefined by our current simulation.
They directly control properties of the cells.
In this example, we focus on a limited subset of parameters which are relevant for our simulation.

In [ ]:
simulation_settings = SimulationSettings()

# Cargo
simulation_settings.n_cells_cargo=250
simulation_settings.particle_template_cargo.interaction.clustering_strength = 0.05
simulation_settings.particle_template_cargo.interaction.potential_strength = 2.0

# ATG11/Receptor
simulation_settings.n_cells_atg11_receptor=400
simulation_settings.particle_template_atg11_receptor.interaction.clustering_strength = 0.05
simulation_settings.particle_template_atg11_receptor.interaction.interaction_range = 2.0
simulation_settings.particle_template_atg11_receptor.mechanics.diffusion_constant = 1.0

# General Settings
simulation_settings.n_threads = 2
simulation_settings.dt = 0.2
simulation_settings.domain_size = 75
simulation_settings.n_times = 1_000

## Actually run the Simulation
Running the simulation is very simple. We have previously defined the settings and will now simply call the run_simulation function.

### Note!
There are many interesting details about how `cellular_raza` works and what you can do with it. For now, these details remain hidden behind this function. If you want a fully flexible simulation framework, you are encouraged to read [the book](https://jonaspleyer.github.io/cellular_raza/).

In [ ]:
output_path = run_simulation(simulation_settings)
# output_path = "out/autophagy/2023-10-31-00:19:21/"
# import os
# from pathlib import Path
# output_path = Path("out/autophagy/") / os.listdir("out/autophagy/")[-1]

## Read results from json Files
The results of the simulation are saved in json files.
Due to the parallelized nature of the simulation, not all results are in one big json file but rather in multiple batches. We therefore need to combine these batches to obtain a complete set for a given iteration.

In [ ]:
import os
from pathlib import Path
import json

def combine_batches(run_directory):
    # Opens all batches in a given directory and stores
    # them in one unified big list
    combined_batch = []
    for batch_file in os.listdir(run_directory):
        f = open(run_directory / batch_file)
        b = json.load(f)["data"]
        combined_batch.extend(b)
    return combined_batch

def get_cells_at_iterations(output_path):
    # Uses the previously defined funtion [combine_batches]
    # to read all stored cells at all iterations and stores
    # them in a dictionary.
    dir = Path(output_path) / "cell_storage/json/"
    runs = [(x, dir / x) for x in os.listdir(dir)]
    result = []
    for (n_run, run_directory) in runs:
        result.extend([{"iteration":int(n_run)} | c for c in combine_batches(run_directory)])
    return result

cells_at_iter = get_cells_at_iterations(output_path)

We want to inspect which entries our generated dataset has. Therefore, we normalize the dict, transforming it into a dataframe.
Afterwards, we display all columns.

In [ ]:
import pandas as pd

df = pd.json_normalize(cells_at_iter)
for col in df.columns:
    print(col)

In [ ]:
max_iter = max(df["iteration"])
df[(df["iteration"]==max_iter) & (df["element.cell.interaction.species"]=="Cargo")]["element.cell.mechanics.mechanics.pos"]

## Plot Result
We visualize the results in 3D.
Therefore we use `pyvista` which internally uses `vtk` as a backend.
Since all of our particles are represented as 3D-spheres, we also display them as such.

In [ ]:
import pyvista as pv
import numpy as np
import multiprocessing as mp

def generate_spheres(df, iteration):
    # Filter for only particles at the specified iteration
    df_filtered = df[df["iteration"]==iteration]

    # Create a dataset for pyvista for plotting
    pset = pv.PolyData(np.array([np.array(x) for x in df_filtered["element.cell.mechanics.mechanics.pos"]]))

    # Extend dataset by species and diameter
    pset.point_data["diameter"] = 2.0*df_filtered["element.cell.interaction.cell_radius"]
    pset.point_data["species"] = df_filtered["element.cell.interaction.species"]

    # Create spheres glyphs from dataset
    sphere = pv.Sphere()
    spheres = pset.glyph(geom=sphere, scale="diameter", orient=False)

    return spheres

def save_snapshot(iteration):
    spheres = generate_spheres(df, iteration)

    spheres.plot(
        off_screen=True,
        screenshot=Path(output_path) / "snapshot_{:08}.png".format(iteration),
        scalars="species",
        scalar_bar_args={
            "title":"Species",
        },
        cpos=[
            (
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size,
                -1.5*simulation_settings.domain_size
            ),(
                25,
                25,
                25
            ),(
                0.0,
                0.0,
                0.0
            )
        ],
        jupyter_backend='none',
    )

We can save single snapshots or even use all processes of our device to save snapshots for every iteration.
The 2nd approach will take up all resources by default. If you want to limit this, have a look at the [Pool object of the multiprocessing](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) module.

In [ ]:
# Save all snapshots
with mp.Pool() as p:
    p.map(save_snapshot, np.unique(df["iteration"]))

In [ ]:
# Only save one snapshot
save_snapshot(0)